# MySQL里的“六度空间游戏”

接第三章的“维基百科六度分隔”问题（即找出一条链接路径，只要点击链接就可以从一个维基词条到另一个维基词条）

下面代码会把“贝肯数”（一个页面与凯文·贝肯词条页面的链接数）不超过6的维基百科页面存储起来：

In [1]:
from bs4 import BeautifulSoup
import re
import pymysql
from urllib.request import urlopen


conn = pymysql.connect(host='127.0.0.1', user='root', passwd=None, db='mysql', charset='utf8')
cur = conn.cursor()
cur.excute("USE wikipedia")

def insertPageIfNotExists(url):
    cur.excute("SELECT *FROM pages WHERE url = %s", (url))
    if cur.rowcount == 0:
        cur.excute("INSERT INTO pages (url) VALUES (%s)", (url))
        conn.commit()
        return cur.lastrowid
    else:
        return cur.fetchone()[0]
    
def insertLink(fromPageId, toPageId):
    cur.excute("SELECT * FROM links WHERE fromPageId = %s AND toPageId = %s", (int(fromPageId), int(toPageId)))
    if cur.rowcount == 0:
        cur.excute("INSERT INTO links (fromPageId, toPageId) VALUES (%s, %s)", (int(fromPageId), int(toPageId)))
        conn.commit()

pages = set()
def getLinks(pageUrl, recursionLevel):
    global pages
    if recursionLevel > 4:
        return
    pageId = insertPageIfNotExists(pageUrl)
    html = urlopen("http://en.wikipedia.org" + pageUrl)
    bsObj = BeautifulSoup(html)
    for link in bsObj.find_all("a", href=re.compile("^(/wiki/)((?!:).)*$")):
        insertLink(pageId, insertPageIfNotExists(link.attrs['href']))
        if link.attrs['href'] not in pages:
            newPage = link.attrs['href']
            pages.add(newPage)
            getLinks(newPage, recursionLevel + 1)

getLinks("/wiki.Kevin_Bacon", 0)
cur.close()
conn.close()

SyntaxError: unexpected EOF while parsing (<ipython-input-1-42095e67380c>, line 12)